In [1]:
import requests
import os

In [2]:
import discord

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
client = discord.Client()

def jisho_lookup(word):
    data = requests.get('https://jisho.org/api/v1/search/words?keyword=' + word).json()
    return data["data"]

def generate_jembed(word):
    if "is_common" not in word:
        return None
    meanings = "";
    pos = "";
    if word["senses"][0]["parts_of_speech"]:
        pos = " (";
        for p in range(len(word["senses"][0]["parts_of_speech"]) - 1):
            pos = pos + word["senses"][0]["parts_of_speech"][p] + ", "
        pos = pos + word["senses"][0]["parts_of_speech"][-1] + ")"
    if word["senses"][0]["english_definitions"]:
        for m in range(len(word["senses"][0]["english_definitions"]) - 1):
            meanings = meanings + word["senses"][0]["english_definitions"][m] + "; "
        meanings = meanings + word["senses"][0]["english_definitions"][-1]
    embed = discord.Embed(title = word["slug"], description = word["japanese"][0]["reading"], color=0x00ff00 if word["is_common"] else 0xffff00)
    embed.add_field(name = "Meanings" + pos, value = meanings, inline=False)
    embed.add_field(name = "Difficulty", value = word["jlpt"][0] if word["jlpt"] else "-", inline=False)
    return embed

@client.event
async def on_ready():
  print("{0} is ready".format(client.user))

@client.event
async def on_message(message):
  if message.author == client.user:
    return
  
  if message.content.startswith('>d '):
    try:
        word = jisho_lookup(message.content[3:].lower())[0]
        em = generate_jembed(word)
        if em:
            await message.channel.send(embed = em)
    except:
        await message.channel.send('Sorry, no results :[ Try splitting up the word(s) or spell things correctly')
  elif message.content.startswith('>da'):
    try:
        words = jisho_lookup(message.content[4:].lower())
        for word in words:
            em = generate_jembed(word)
            if em:
                await message.channel.send(embed = em)
    except IndexError:
        await message.channel.send('Sorry, no results :[ Try splitting up the word(s) or spell things correctly')
  elif message.content.startswith('>d'):
    parts = message.content.split(" ", 1)
    n = int(parts[0][2:])
    if n>0:
        try:
            words = jisho_lookup(parts[1].lower())
            for word in words:
                if n<=0:
                    break
                em = generate_jembed(word)
                if em:
                    await message.channel.send(embed = em)
                    n = n - 1
        except IndexError:
            await message.channel.send('Sorry, no results :[ Try splitting up the word(s) or spell things correctly')
    else:
        await message.channel.send("Bruh")
  if message.content.startswith('>h'):
    em = discord.Embed(title="SaFL Bot Commands", description="Find out what I can do!", color=0x0000ff)
    em.add_field(name="Dictionary",
                 value="`>d [word]` for the first result\n`>dn [word]` for the first n results\n`>da [word]` for all results. *WARNING: Might cause spam. Use sparingly*\n\nTip: Green border? Commonly used word! Yellow? Not so much. Try peeking at the first 2 results of \"kazoku\"  ( `>d2 kazoku` )",
                 inline=False)
    await message.channel.send(embed=em)

client.run('secret-key')

SaFL#8618 is ready
